In [5]:
import polars as pl

from datetime import datetime

from io import StringIO

In [2]:
"""
Notes for the users_data_df:--> done
- addresses need to be split out--> Done
- then see if you can take the per_capita_income and make it more usefull --> Unfortunately means nothing --> done
- remove num_credit_cards --> done

Notes for cards_data_df: --> done
- do historization layer, although there is close_date (maybe with year_pin_last_changed) --> Done


Notes for transactions_data_df:
- Join the erros into this to capture them, no need to split them separately
- join out the merchant things

Notes for new Merchant_Table:
- contains the merchant_id and merchant_address and mcc_code (merchant_category)
- has 2 separate tables:
- Merchant_category and MErchant_Address


Notes for Merchant_address_table:
- one merchant can have multiple addresses
- so the Primary key is a structured key
- there is no historization (if addresses change there will be new entries)


Notes for the Client_address table: --> Done
- create it and historize it
- take out the number form the address and split it into house-number
"""

'\nNotes for the users_data_df:--> done\n- addresses need to be split out--> Done\n- then see if you can take the per_capita_income and make it more usefull --> Unfortunately means nothing --> done\n- remove num_credit_cards --> done\n\nNotes for cards_data_df: --> done\n- do historization layer, although there is close_date (maybe with year_pin_last_changed) --> Done\n\n\nNotes for transactions_data_df:\n- Join the erros into this to capture them, no need to split them separately\n- join out the merchant things\n\nNotes for new Merchant_Table:\n- contains the merchant_id and merchant_address and mcc_code (merchant_category)\n- has 2 separate tables:\n- Merchant_category and MErchant_Address\n\n\nNotes for Merchant_address_table:\n- one merchant can have multiple addresses\n- so the Primary key is a structured key\n- there is no historization (if addresses change there will be new entries)\n\n\nNotes for the Client_address table: --> Done\n- create it and historize it\n- take out the n

## Create LazyFrames for all Files

In [10]:
# CSVs
cards_data_lf = pl.scan_csv(source=r"C:\Users\Jan\programming\data_modelling_1\data\archive\cards_data.csv")
transactions_data_lf = pl.scan_csv(r"C:\Users\Jan\programming\data_modelling_1\data\archive\transactions_data.csv")
users_data_lf = pl.scan_csv(r"C:\Users\Jan\programming\data_modelling_1\data\archive\users_data.csv")
# JSONS
mcc_codes_lf = pl.scan_ndjson(source=r"C:\Users\Jan\programming\data_modelling_1\data\archive\mcc_codes.json" , schema={"mcc":pl.datatypes.String,"merchant_category":pl.datatypes.String})
train_fraud_labels_lf = pl.scan_ndjson(source=r"C:\Users\Jan\programming\data_modelling_1\data\archive\mcc_codes.json")

In [11]:
train_fraud_labels_lf.collect()

ComputeError: Syntax at character 0 ('{')

## Creaete users_data table

In [ ]:
users_data_lf_lz = users_data_lf.select([
                      pl.col("id").alias("client_id")
                      ,"current_age"
                      ,"retirement_age"
                      ,"birth_year"
                      ,"birth_month" 
                      ,"gender"
                      ,pl.int_range(pl.len(),dtype=pl.UInt32).alias("address_id")
                      ,"address"
                      ,"per_capita_income"
                      ,"yearly_income"
                      ,"total_debt"
                      ,"credit_score"
                      ,pl.lit(datetime.now() , dtype=pl.Date).alias("valid_from")
                      ,pl.date(9999,12,31).alias("valid_to")
                     ]
                    )

## Create CLIENT_ADRESS Table

In [ ]:
# Creation of CLIENT_ADDRESS Table
client_address = users_data_lf_lz.select(["address_id" , "address"])
client_address = client_address.with_columns(
    pl.col("address").str.split(" ").list.to_struct().alias("split_address")
    ,pl.lit(datetime.now() , dtype=pl.Date).alias("valid_from")
    ,pl.date(9999,12,31).alias("valid_to")
).unnest(columns="split_address").collect()

client_address = pl.LazyFrame(client_address.select(
    "address_id"
    ,pl.col("field_0").alias("house_number")
    ,pl.concat_str([pl.col("field_1"),pl.col("field_2"),],separator=" ").alias("street_name")
    ,"valid_from"
    ,"valid_to"
))

## Create cards_data

In [ ]:
cards_data_lf = cards_data_lf.with_columns(
    pl.lit(datetime.now() , dtype=pl.Date).alias("valid_from")
    ,pl.date(9999,12,31).alias("valid_to")
)

## Create MERCHANT_ADDRESS table

In [ ]:
merchant_address_lf = transactions_data_lf.select(
    "merchant_id"
    ,pl.int_range(pl.len(),dtype=pl.UInt32).alias("merchant_address_id")
    ,"merchant_city"
    ,"merchant_state"
    ,"zip"
    ,pl.lit("Y").alias("active_flag")
)

## Create transactions table

In [ ]:
transactions_data_lf.collect()

In [ ]:
transactions_data_lf.select(
    "id"
    ,pl.col("date").str.strptime(pl.Datetime).alias("date")
    ,"client_id"
    ,"card_id"
    ,pl.col("amount").str.replace("$","")
    ,"use_chip"
    ,"merchant_id"
    ,pl.int_range(pl.len(),dtype=pl.UInt32).alias("merchant_address_id")
    ,"mcc"
    ,"errors"
).cast({"date":pl.Datetime}).collect().join(mcc_codes_lf,on="")

In [ ]:
transactions_data_lf
merchant_dim = 1